In [44]:
import pyspark

In [45]:
from pyspark.sql import SparkSession

In [46]:
spark = SparkSession.builder.appName("SocialNetworkAnalysis").config("spark.jars", "C:/Big Data/graphframes/graphframes-0.8.2-spark3.0-s_2.12.jar").config("spark.driver.memory", "8g").config("spark.executor.memory", "8g").getOrCreate()

In [47]:
spark.conf.set("spark.sql.shuffle.partitions", "200")

In [48]:
spark

In [49]:
spark.sparkContext.setCheckpointDir(r"C:\Master-Projects\Social_Media_BigData\checkpoints")

In [50]:
df = spark.read.text("facebook_combined.txt.gz")

In [51]:
df.show()

+-----+
|value|
+-----+
|  0 1|
|  0 2|
|  0 3|
|  0 4|
|  0 5|
|  0 6|
|  0 7|
|  0 8|
|  0 9|
| 0 10|
| 0 11|
| 0 12|
| 0 13|
| 0 14|
| 0 15|
| 0 16|
| 0 17|
| 0 18|
| 0 19|
| 0 20|
+-----+
only showing top 20 rows



In [52]:
from pyspark.sql.functions import split, col
from graphframes import GraphFrame

In [53]:
edges = df.withColumn("src", split(col("value"), " ")[0]).withColumn("dst", split(col("value"), " ")[1]).select("src", "dst")

In [54]:
vertices = edges.select(col("src").alias("id")).union(edges.select(col("dst").alias("id"))).distinct()

In [55]:
graph = GraphFrame(vertices, edges)

* a) Si mund të identifikojmë përdoruesit më të ndikshëm duke përdorur algoritme të centralitetit?

In [56]:
# Apliko PageRank
pagerank = graph.pageRank(resetProbability=0.15, maxIter=10)
pagerank.vertices.orderBy(col("pagerank").desc()).show(10, truncate=False)

+----+------------------+
|id  |pagerank          |
+----+------------------+
|1911|40.173020035189126|
|3434|38.21196968977279 |
|2655|37.630247553347424|
|1902|37.243511110381164|
|1888|28.028255649535605|
|2649|25.511310917057735|
|1907|21.240971021534005|
|3971|20.558757839388395|
|2654|20.20063587309717 |
|1910|17.551329484911413|
+----+------------------+
only showing top 10 rows



b) Si mund të përdoren grafët për të zbuluar grupe të ngjashme përdoruesish në një rrjet social?

In [41]:
graph.edges.show()

+---+---+
|src|dst|
+---+---+
|  0|  1|
|  0|  2|
|  0|  3|
|  0|  4|
|  0|  5|
|  0|  6|
|  0|  7|
|  0|  8|
|  0|  9|
|  0| 10|
|  0| 11|
|  0| 12|
|  0| 13|
|  0| 14|
|  0| 15|
|  0| 16|
|  0| 17|
|  0| 18|
|  0| 19|
|  0| 20|
+---+---+
only showing top 20 rows



In [42]:
graph.vertices.show()

+----+
|  id|
+----+
| 467|
| 675|
| 691|
| 829|
|1090|
|1159|
|1436|
|1512|
|1572|
|2069|
|2088|
|2136|
|2162|
|2294|
|2904|
|3210|
|3606|
| 125|
| 451|
| 800|
+----+
only showing top 20 rows



In [57]:
connected_components = graph.connectedComponents()

Py4JJavaError: An error occurred while calling o277.run.
: java.lang.OutOfMemoryError: Java heap space


In [ ]:
# Gjej komponentët e lidhur
spark.sparkContext.setCheckpointDir(r"C:\Big Data\checkpoint")
graph = GraphFrame(vertices, edges)
connected_components = graph.connectedComponents()
connected_components.orderBy("component").show(10, truncate=False)

c) Si mund të përdorim algoritmet e përhapjes së informacionit për të analizuar shpërndarjen e lajmeve ose viruseve?

In [ ]:
# Apliko algoritmin e përhapjes së informacionit
label_propagation = graph.labelPropagation(maxIter=5)
label_propagation.orderBy("label").show(10, truncate=False)

d) Çfarë ndikimi ka heqja e një përdoruesi kyç në strukturën dhe stabilitetin e rrjetit social?

In [ ]:
# Hiq një përdorues nga grafi
user_to_remove = "0"  # P.sh., përdoruesi me ID 0
updated_edges = edges.filter((col("src") != user_to_remove) & (col("dst") != user_to_remove))
updated_vertices = vertices.filter(col("id") != user_to_remove)
updated_graph = GraphFrame(updated_vertices, updated_edges)

# Analizo strukturën e re
updated_graph.connectedComponents().show()

e) Si mund të përdorim PageRank për të vlerësuar ndikimin e përdoruesve në një rrjet social?